<a href="https://colab.research.google.com/github/mikeguzman1294/OperationsResearch/blob/main/HubLocationNetworkOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hub Location Project 

Authors:
1.   Miguel Angel Guzman Sanchez
2.   Kadriye Nur Bakirci




## Implementation of the HUB location 

### Prepare the Environment and Data

In [162]:
# Clone the repo containing the dataset
!git clone -l -s https://github.com/mikeguzman1294/OperationsResearch.git cloned-repo
%cd cloned-repo/Datasets

Cloning into 'cloned-repo'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 8), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.
/content/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo/Datasets


In [140]:
# Install pulp
!pip install pulp

In [141]:
# Import Required Libraries
import pandas as pd
from pulp import *

In [164]:
# Define the path for importing the dataset
InputData = '/content/cloned-repo/Datasets/InputDataHubSmallInstance.xlsx'

# Define a function to extract the information from the dataset
def read_excel_data(filename, sheet_name):
  data = pd.read_excel(filename, sheet_name=sheet_name, header=None)
  values = data.values
  if min(values.shape) == 1:  # This If is to make the code insensitive to column-wise or row-wise expression #
      if values.shape[0] == 1:
          values = values.tolist()
      else:
          values = values.transpose()
          values = values.tolist()
      return values[0]        
  else:
      data_dict = {}
      if min(values.shape) == 2:  # For single-dimension parameters in Excel
          if values.shape[0] == 2:
              for i in range(values.shape[1]):
                  data_dict[i+1] = values[1][i]
          else:
              for i in range(values.shape[0]):
                  data_dict[i+1] = values[i][1]
             
      else:  # For two-dimension (matrix) parameters in Excel
          for i in range(values.shape[0]):
              for j in range(values.shape[1]):
                  data_dict[(i+1, j+1)] = values[i][j]
      return data_dict

In [143]:
# NodeNum sheet contains the number of nodes in the network
# Since it only contains 1 element the function treats it as one-dimensional array (list)
# First and only index is retrieved to just save an integer value
node_num = read_excel_data(InputData, "NodeNum")[0]

# The sets of the function can be built out of this information
# Create the set of nodes
set_nodes = list(range(1,node_num+1))
print(set_nodes)

[1, 2, 3, 4, 5, 6, 7, 8]


In [144]:
# Fixed cost of locating a hub at node 
fix_cost = read_excel_data(InputData, "fixCost(fk)")
print("FixCost: ", fix_cost)

FixCost:  [325478, 420241, 324187, 324869, 532481, 387526, 334862, 388719]


In [145]:
# Capacity of hub k
cap = read_excel_data(InputData, "Cap(ckmax)")
print("Cap: ", cap)

Cap:  [95231, 137521, 62418, 61486, 83248, 131752, 63486, 98871]


In [146]:
# NodeNum sheet contains the Hub-to-hub discount factor (0 < α < 1)
# Since it only contains 1 element the function treats it as one-dimensional array (list)
# First and only index is retrieved to just save an integer value
alpha = read_excel_data(InputData, "alpha")[0]
print("Alpha: ", alpha)

Alpha:  0.65


In [147]:
# Variable transfer cost of flow through the link from node i to node j
var_cost = read_excel_data(InputData, "varCost(cij)")
print("VarCost: ", var_cost)

VarCost:  {(1, 1): 0, (1, 2): 32, (1, 3): 29, (1, 4): 30, (1, 5): 10, (1, 6): 15, (1, 7): 28, (1, 8): 17, (2, 1): 32, (2, 2): 0, (2, 3): 16, (2, 4): 33, (2, 5): 38, (2, 6): 25, (2, 7): 17, (2, 8): 23, (3, 1): 29, (3, 2): 16, (3, 3): 0, (3, 4): 32, (3, 5): 40, (3, 6): 27, (3, 7): 24, (3, 8): 26, (4, 1): 30, (4, 2): 33, (4, 3): 32, (4, 4): 0, (4, 5): 19, (4, 6): 23, (4, 7): 31, (4, 8): 27, (5, 1): 10, (5, 2): 38, (5, 3): 40, (5, 4): 19, (5, 5): 0, (5, 6): 33, (5, 7): 17, (5, 8): 32, (6, 1): 15, (6, 2): 25, (6, 3): 27, (6, 4): 23, (6, 5): 33, (6, 6): 0, (6, 7): 38, (6, 8): 10, (7, 1): 28, (7, 2): 17, (7, 3): 24, (7, 4): 31, (7, 5): 17, (7, 6): 38, (7, 7): 0, (7, 8): 30, (8, 1): 17, (8, 2): 23, (8, 3): 26, (8, 4): 27, (8, 5): 32, (8, 6): 10, (8, 7): 30, (8, 8): 0}


In [148]:
# Amount of flow from node i to node j
flow = read_excel_data(InputData, "flow(wij)")
print("flow: ", flow)

flow:  {(1, 1): 0, (1, 2): 3032, (1, 3): 1815, (1, 4): 3364, (1, 5): 2969, (1, 6): 1698, (1, 7): 2611, (1, 8): 3597, (2, 1): 1276, (2, 2): 0, (2, 3): 3371, (2, 4): 2437, (2, 5): 1371, (2, 6): 3094, (2, 7): 3700, (2, 8): 3456, (3, 1): 4022, (3, 2): 3984, (3, 3): 0, (3, 4): 3360, (3, 5): 2420, (3, 6): 4107, (3, 7): 2176, (3, 8): 3091, (4, 1): 1719, (4, 2): 3689, (4, 3): 3401, (4, 4): 0, (4, 5): 1905, (4, 6): 2831, (4, 7): 2269, (4, 8): 4196, (5, 1): 3902, (5, 2): 3548, (5, 3): 3174, (5, 4): 2464, (5, 5): 0, (5, 6): 4044, (5, 7): 3839, (5, 8): 2644, (6, 1): 3474, (6, 2): 3345, (6, 3): 1694, (6, 4): 1822, (6, 5): 4148, (6, 6): 0, (6, 7): 1829, (6, 8): 2749, (7, 1): 2978, (7, 2): 3627, (7, 3): 3324, (7, 4): 3104, (7, 5): 4020, (7, 6): 4194, (7, 7): 0, (7, 8): 2450, (8, 1): 3535, (8, 2): 2665, (8, 3): 2485, (8, 4): 2986, (8, 5): 3930, (8, 6): 1686, (8, 7): 4015, (8, 8): 0}


In [149]:
# Computation of Oi
# Oi stands for total flow originating from node i
oi = [0] * (len(set_nodes))

for row in range(1, len(set_nodes)+1):
  for column in range(1, len(set_nodes)+1):
    oi[row-1] += flow[(row, column)]

print(oi)


[19086, 18705, 23160, 20010, 23615, 19061, 23697, 21302]


In [150]:
# Computation of Di
# Di standa for total flow destination in node i
di = [0] * (len(set_nodes))

for column in range(1, len(set_nodes)+1):
  for row in range(1, len(set_nodes)+1):
    di[column-1] += flow[(row,column)]

print(di)

[20906, 23890, 19264, 19537, 20763, 21654, 20439, 22183]


### Creating the Model

#### Decision Variables and Objective Function

In [151]:
# Define decision variables
y_var = LpVariable.dicts('y', (set_nodes, set_nodes), lowBound=0, upBound=1, cat='Binary')
print(y_var)

z_var = LpVariable.dicts('z', (set_nodes, set_nodes), lowBound=0, upBound=1, cat='Binary')
print(z_var)

x_var = LpVariable.dicts('x', (set_nodes, set_nodes, set_nodes), lowBound=0, cat='Continuous')
print(x_var)

{1: {1: y_1_1, 2: y_1_2, 3: y_1_3, 4: y_1_4, 5: y_1_5, 6: y_1_6, 7: y_1_7, 8: y_1_8}, 2: {1: y_2_1, 2: y_2_2, 3: y_2_3, 4: y_2_4, 5: y_2_5, 6: y_2_6, 7: y_2_7, 8: y_2_8}, 3: {1: y_3_1, 2: y_3_2, 3: y_3_3, 4: y_3_4, 5: y_3_5, 6: y_3_6, 7: y_3_7, 8: y_3_8}, 4: {1: y_4_1, 2: y_4_2, 3: y_4_3, 4: y_4_4, 5: y_4_5, 6: y_4_6, 7: y_4_7, 8: y_4_8}, 5: {1: y_5_1, 2: y_5_2, 3: y_5_3, 4: y_5_4, 5: y_5_5, 6: y_5_6, 7: y_5_7, 8: y_5_8}, 6: {1: y_6_1, 2: y_6_2, 3: y_6_3, 4: y_6_4, 5: y_6_5, 6: y_6_6, 7: y_6_7, 8: y_6_8}, 7: {1: y_7_1, 2: y_7_2, 3: y_7_3, 4: y_7_4, 5: y_7_5, 6: y_7_6, 7: y_7_7, 8: y_7_8}, 8: {1: y_8_1, 2: y_8_2, 3: y_8_3, 4: y_8_4, 5: y_8_5, 6: y_8_6, 7: y_8_7, 8: y_8_8}}
{1: {1: z_1_1, 2: z_1_2, 3: z_1_3, 4: z_1_4, 5: z_1_5, 6: z_1_6, 7: z_1_7, 8: z_1_8}, 2: {1: z_2_1, 2: z_2_2, 3: z_2_3, 4: z_2_4, 5: z_2_5, 6: z_2_6, 7: z_2_7, 8: z_2_8}, 3: {1: z_3_1, 2: z_3_2, 3: z_3_3, 4: z_3_4, 5: z_3_5, 6: z_3_6, 7: z_3_7, 8: z_3_8}, 4: {1: z_4_1, 2: z_4_2, 3: z_4_3, 4: z_4_4, 5: z_4_5, 6: z_4_6,

In [152]:
# Setting the Problem
hub_problem = LpProblem("Capacitated_Facility_Location_Problem", LpMinimize)

In [153]:
# Defining objective function
# We add the fixed cost
hub_problem += lpSum(fix_cost[k-1] * z_var[k][k] for k in set_nodes) + \
               lpSum( ( (var_cost[(i,k)] * oi[i-1])  + (var_cost[(k,i)] * di[i-1])  ) * z_var[i][k]  for i in set_nodes for k in set_nodes) + \
               lpSum( ( (alpha*var_cost[(k,l)]*x_var[i][k][l] for i in set_nodes for k in set_nodes for l in set_nodes if l != k) ) )

#### Adding the constraints

In [154]:
# Constraint 1
for i in set_nodes:
  hub_problem += lpSum(z_var[i][k] for k in set_nodes) == 1

In [155]:
# Constraint 2
for k in set_nodes:
  for l in set_nodes:
    if l > k:
      hub_problem += z_var[k][l] + y_var[k][l] <= z_var[l][l]
      hub_problem += z_var[l][k] + y_var[k][l] <= z_var[k][k]

In [156]:
# Constraint 3
for i in set_nodes:
  for k in set_nodes:
    for l in set_nodes:
      if l > k:
        hub_problem += x_var[i][k][l] + x_var[i][l][k] <= oi[i-1]*y_var[k][l]

In [157]:
# Constraint 4
for i in set_nodes:
  for k in set_nodes:
    if i != k:
      hub_problem += oi[i-1] * z_var[i][k] + lpSum( x_var[i][l][k] for l in set_nodes if l != k) == \
                     lpSum( x_var[i][k][l] for l in set_nodes if l != k) + lpSum( flow[(i,l)] * z_var[l][k] for l in set_nodes)

In [158]:
# Constraint 5
for k in set_nodes:
  hub_problem += lpSum( (x_var[i][l][k] for i in set_nodes for l in set_nodes) ) + lpSum(oi[i-1] * z_var[i][k] for i in set_nodes) <= cap[k-1]

In [159]:
# Constraint 6
hub_problem += lpSum( (y_var[k][l] for k in set_nodes for l in set_nodes) ) == lpSum(z_var[k][k] for k in set_nodes) - 1

#### Solving the model with default solver of PuLP

In [160]:
# The problem is solved using PuLP's choice of Solver
# (the default solver is Coin Cbc)
hub_problem.solve()

1

In [161]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[hub_problem.status])

# The optimal value of the decision variables and the
# optimised objective function value is printed to the screen
#for v in hub_problem.variables():
    #print(v.name, "=", v.varValue)

print ("Objective value HUB Network Optimization Problem = ", value(hub_problem.objective))

Status: Optimal
Objective value HUB Network Optimization Problem =  6794788.65
